In [1]:
import pandas as pd
from fuzzywuzzy import fuzz
import re
from nltk import sent_tokenize
from nltk import ngrams
from joblib import Parallel, delayed

c:\Users\giang\anaconda3\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
def preprocess_text(text):
    """Preprocess the text: convert to lowercase, replace punctuation with space, and reduce multiple spaces to one."""
    text = text.lower()  # Chuyển văn bản thành chữ thường
    text = re.sub(r'[^\w\s]', ' ', text)  # Thay thế dấu câu bằng khoảng trắng
    text = re.sub(r'\s+', ' ', text)  # Rút gọn nhiều khoảng trắng thành một
    text = text.strip()  # Loại bỏ khoảng trắng đầu và cuối
    return text

In [3]:
def generate_ngrams(text, n):
    """Generate n-grams from the text."""
    words = text.split()
    return [' '.join(gram) for gram in ngrams(words, n)]

In [4]:
def check_group(sentences, keywords, threshold=90):
    """Check if the group of keywords is related to the abstract."""
    all_ngrams = []
    for sentence in sentences:
        for n in range(1, 5):  # From 1-gram to 4-gram
            all_ngrams.extend(generate_ngrams(sentence, n))
    
    for keyword in keywords:
        for ngram in all_ngrams:
            if fuzz.WRatio(keyword, ngram) > threshold:
                return True
        for sentence in sentences:
            if fuzz.WRatio(keyword, sentence) > threshold:
                return True
    return False

In [ ]:
def classify_abstract(abstract):
    """Classify the abstract and return a list of True/False for each keyword group."""
    abstract = preprocess_text(abstract)
    sentences = sent_tokenize(abstract)
    
    text_to_sql_keywords = ["functional testing", "LLM", "LLM to Funtional test", "Test Generation", "Automation generative", "UI Testing", "Benchmark for functional testing"]
    security_keywords = ["security", "access control", "injection", "prompt injection", 
                         "defense", "attack", "vulnerability"]
    llm_keywords = ["llm", "large language model"]
    
    t2sql_result = check_group(sentences, text_to_sql_keywords)
    security_result = check_group(sentences, security_keywords)
    llm_result = check_group(sentences, llm_keywords)
    
    return [t2sql_result, security_result, llm_result]

In [6]:
def process_csv(input_csv, output_csv=None):
    """
    Đọc file CSV, phân loại từng abstract với xử lý song song, và thêm 3 cột boolean mới: t2sql, security, llm.
    
    :param input_csv: Đường dẫn đến file CSV đầu vào.
    :param output_csv: Đường dẫn để lưu file CSV đầu ra. Nếu None, không lưu.
    :return: DataFrame đã được cập nhật với các cột boolean mới.
    """
    df = pd.read_csv(input_csv)
    
    if 'abstract' not in df.columns:
        raise ValueError("File CSV phải chứa cột 'abstract'.")
    
    df['abstract'] = df['abstract'].astype(str)
    classifications = Parallel(n_jobs=-1)(delayed(classify_abstract)(abstract) for abstract in df['abstract'])
    
    df['t2sql'] = [result[0] for result in classifications]
    df['security'] = [result[1] for result in classifications]
    df['llm'] = [result[2] for result in classifications]
    
    df['submitted'] = pd.to_datetime(df['submitted'])
    df = df.sort_values(by='submitted', ascending=False)

    if output_csv:
        df.to_csv(output_csv, index=False)
    
    return df

In [8]:
type = "ieee"
input_csv = f"../raw_crawl_papers/date_processed/{type}_papers.csv"  # Thay bằng đường dẫn file CSV của bạn
output_csv = f"../keywords_summary/{type}/{type}_with_keywords.csv"  # Thay bằng đường dẫn file CSV đầu ra
df = process_csv(input_csv, output_csv)
print(f"Total papers: {len(df)}")

Total papers: 126


In [9]:
# 1. Trích xuất các record có True trong cả 3 cột
all_true_df = df[(df['t2sql'] == True) & (df['llm'] == True) & (df['security'] == True)]
print(f"Total number of papers that relating to t2sql, llm and security keywords: {len(all_true_df)}")

all_true_df = all_true_df.copy()

all_true_df['submitted'] = pd.to_datetime(all_true_df['submitted'])
all_true_df = all_true_df.sort_values(by='submitted', ascending=False)

all_true_df.to_csv(f"../keywords_summary/{type}/all_true.csv", index=False)

Total number of papers that relating to t2sql, llm and security keywords: 2


In [10]:
# 2. Trích xuất các record có True trong t2sql và llm
t2sql_llm_true_df = df[(df['t2sql'] == True) & (df['llm'] == True)]
print(f"Total number of papers that relating to t2sql and llm keywords: {len(t2sql_llm_true_df)}")

t2sql_llm_true_df = t2sql_llm_true_df.copy()

t2sql_llm_true_df['submitted'] = pd.to_datetime(t2sql_llm_true_df['submitted'])
t2sql_llm_true_df = t2sql_llm_true_df.sort_values(by='submitted', ascending=False)

t2sql_llm_true_df.to_csv(f"../keywords_summary/{type}/t2sql_llm_true.csv", index=False)

Total number of papers that relating to t2sql and llm keywords: 43


In [11]:
# 3. Trích xuất các record có True trong t2sql và security
t2sql_security_true_df = df[(df['t2sql'] == True) & (df['security'] == True)]
print(f"Total number of papers that relating to t2sql and security keywords: {len(t2sql_security_true_df)}")

t2sql_security_true_df = t2sql_security_true_df.copy()

t2sql_security_true_df['submitted'] = pd.to_datetime(t2sql_security_true_df['submitted'])
t2sql_security_true_df = t2sql_security_true_df.sort_values(by='submitted', ascending=False)

t2sql_security_true_df.to_csv(f"../keywords_summary/{type}/t2sql_security_true.csv", index=False)

Total number of papers that relating to t2sql and security keywords: 4


In [12]:
# Hợp nhất ba DataFrame
merged_df = pd.concat([all_true_df, t2sql_llm_true_df, t2sql_security_true_df], ignore_index=True)

# Loại bỏ các bản ghi trùng lặp
merged_df = merged_df.drop_duplicates()

# Lưu vào tệp CSV mới
output_path = f"../keywords_summary/{type}/merged_keyword_results.csv"
merged_df.to_csv(output_path, index=False)

# In số lượng bản ghi trong DataFrame hợp nhất
print(f"Total number of unique papers in merged DataFrame: {len(merged_df)}")

Total number of unique papers in merged DataFrame: 45
